<a href="https://colab.research.google.com/github/Fairuza12/MSProject/blob/main/T5_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import difflib

In [2]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 17.1 MB/s eta 0:00:00


In [3]:
import easyocr
import pandas as pd
import os

In [4]:
from transformers import pipeline

In [5]:
# Initialize EasyOCR for Bangla
reader = easyocr.Reader(['bn'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [6]:
# Initialize pretrained Bangla T5 model for text refinement
text_correct_model = pipeline("text2text-generation", model = "csebuetnlp/banglat5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
# Read the Excel file mapping images to annotations
df = pd.read_excel("/content/drive/MyDrive/MS_Project/Datasets/HCR_DATASET/VHD/Image_text_Mapping_1.xlsx")

# Folder paths
image_folder = "/content/drive/MyDrive/MS_Project/Datasets/HCR_DATASET/VHD/images/"
annotations_folder = "/content/drive/MyDrive/MS_Project/Datasets/HCR_DATASET/VHD/annotations/"

In [8]:
def correct_text_with_t5(ocr_text):
  try:
    result = text_correct_model(ocr_text,max_length = 512)
    return result[0]['generated_text']
  except Exception as e:
    print(f"Error in text correction: {e}")
    return ocr_text

In [9]:
# OCR function for text correction
def ocr_and_correct(image_path):
  # Use EasyOCR to extract text from the image
  results = reader.readtext(image_path)
  # Extract that text
  ocr_text = ' '.join([text for (bbox,text,prob) in results])
  # Use the Bangla T5 model to improve the OCR text
  corrected_text = correct_text_with_t5(ocr_text)
  return corrected_text

In [10]:
def calculate_accuracy(predicted,actual):
  return difflib.SequenceMatcher(None,predicted,actual).ratio()

In [11]:
# Process dataset and calculate accuracy for increasing dataset sizes
def process_dataset(df_subset):

  total_accuracy = 0
  count = 0

  for index,row in df_subset.iterrows():
    image_name = row['Image']
    text_file = row['Text']

    if pd.isna(text_file) or 'N/A' in text_file:
      continue

    image_path = os.path.join(image_folder,image_name)
    annotation_path = os.path.join(annotations_folder,text_file)

    # Read ground truth text
    with open(annotation_path,'r',encoding = 'utf-8') as f:
      ground_truth = f.read().strip()

    # Perform OCR and correction
    predicted_text = ocr_and_correct(image_path)

    # Calculate accuracy for this image
    accuracy = calculate_accuracy(predicted_text, ground_truth)
    total_accuracy += accuracy
    count += 1

  # Return average accuracy
  return total_accuracy/count

In [ ]:
# Dataset sizes to test with
dataset_sizes = [20, 25, 50, 100, 150, 200]
accuracy_results = []

# Loop over dataset sizes
for size in dataset_sizes:
    df_subset = df[:size]  # Select the first 'size' number of rows
    accuracy = process_dataset(df_subset)
    accuracy_results.append((size, accuracy))
    print(f"Dataset size: {size}, Accuracy: {accuracy * 100:.2f}%")

Dataset size: 20, Accuracy: 1.54%
